In [2]:
import pprint as pp
from kubernetes import client, config

In [3]:

def extracted_pods_data(k8s_client: client.CoreV1Api):
    all_namespases_pods: list = k8s_client.list_pod_for_all_namespaces().to_dict().get('items')
    all_pods_extract_data = []
    for pod in all_namespases_pods:
        pod_volumes = []
        
        # crean not extis volumes
        for volume in pod['spec']['volumes']:
            instance_volume = {
                'name': volume['name']
            }
            for key, value in volume.items():
                if value is not None:
                    instance_volume[key] = value
            pod_volumes.append(instance_volume)
            
        all_pods_extract_data.append( {
            'name': pod['metadata']['name'],
            'namespace': pod['metadata']['namespace'],
            'labels': pod['metadata']['labels'],
            'volume': pod_volumes,
        })
    return all_pods_extract_data

In [4]:
config.load_kube_config()
k8s_client = client.CoreV1Api()
    
all_pods_extract_data = extracted_pods_data(k8s_client)
pp.pprint(all_pods_extract_data[-1])
pp.pprint(all_pods_extract_data[-2])

{'labels': {'app': 'local-path-provisioner', 'pod-template-hash': '988d74bc'},
 'name': 'local-path-provisioner-988d74bc-h9qc5',
 'namespace': 'local-path-storage',
 'volume': [{'config_map': {'default_mode': 420,
                            'items': None,
                            'name': 'local-path-config',
                            'optional': None},
             'name': 'config-volume'},
            {'name': 'kube-api-access-8nznk',
             'projected': {'default_mode': 420,
                           'sources': [{'cluster_trust_bundle': None,
                                        'config_map': None,
                                        'downward_api': None,
                                        'secret': None,
                                        'service_account_token': {'audience': None,
                                                                  'expiration_seconds': 3607,
                                                                  'path': 'token

# Hendel Projected Volume

In [5]:
pod_semple = {'labels': {'app': 'local-path-provisioner', 'pod-template-hash': '988d74bc'},
 'name': 'local-path-provisioner-988d74bc-dq25g',
 'namespace': 'local-path-storage',
 'volume': [{'config_map': {'default_mode': 420,
                            'items': None,
                            'name': 'local-path-config',
                            'optional': None},
             'name': 'config-volume'},
            {'name': 'kube-api-access-hkv6c',
             'projected': {'default_mode': 420,
                           'sources': [{'cluster_trust_bundle': None,
                                        'config_map': None,
                                        'downward_api': None,
                                        'secret': None,
                                        'service_account_token': {'audience': None,
                                                                  'expiration_seconds': 3607,
                                                                  'path': 'token'}},
                                       {'cluster_trust_bundle': None,
                                        'config_map': {'items': [{'key': 'ca.crt',
                                                                  'mode': None,
                                                                  'path': 'ca.crt'}],
                                                       'name': 'kube-root-ca.crt',
                                                       'optional': None},
                                        'downward_api': None,
                                        'secret': None,
                                        'service_account_token': None},
                                       {'cluster_trust_bundle': None,
                                        'config_map': None,
                                        'downward_api': {'items': [{'field_ref': {'api_version': 'v1',
                                                                                  'field_path': 'metadata.namespace'},
                                                                    'mode': None,
                                                                    'path': 'namespace',
                                                                    'resource_field_ref': None}]},
                                        'secret': None,
                                        'service_account_token': None}]}}]}




In [6]:
# Display Pods graph

In [15]:
# Create Pod leve graph 
import itertools
import networkx as nx
from pyvis import network as net
import uuid

random_uuid = uuid.uuid4()

g = net.Network(notebook=True,cdn_resources="in_line",bgcolor="#222222", font_color="white") # Use this set of params in the service
# g.show_buttons(filter_=['physics'])

# nxg = nx.complete_graph(10)
nxg = nx.Graph()

green = '#00e673'
volume_color = '#33bbff'

volume_img_url = "https://github.com/kubernetes/community/blob/master/icons/png/resources/labeled/vol-128.png?raw=true"
pod_img_url = "https://github.com/kubernetes/community/blob/master/icons/png/resources/labeled/pod-128.png?raw=true"

node_size=20


volume_count = {}
import pprint as pp
def generate_pod_graph(pods_data):
    for pod in pods_data:
        pod_name = pod.get('name')
        nxg.add_node(pod_name,label=f"{''.join(pod_name.split('-')[:-1])}",shape='image', image = pod_img_url ,kind="pod",size=node_size)
        for volume in pod.get('volume'):
            # volume_name = f"{volume.get('name')}-{str(random_uuid)}"
            volume_name = f"{volume.get('name')}"

            if 'projected' in volume.keys():
                # print(pod_name)
                nxg.add_node(volume_name,label=f"Projected \n{volume_name}",shape='image', image = volume_img_url,kind="volume",size=node_size)
                nxg.add_edge(pod_name,volume_name,color=green) 
                projected_data = volume['projected']['sources']
                projected_volumes = [ [k for k,v in proj_volume.items() if v != None] for proj_volume in projected_data]
                projected_volumes = list(itertools.chain(*projected_volumes))
                for projected_volume in projected_volumes:
                    nxg.add_node(f"{volume_name}-{projected_volume}",label=f"{projected_volume}",shape='image', image = volume_img_url,kind="volume",size=node_size)
                    nxg.add_edge(volume_name,f"{volume_name}-{projected_volume}",color=green)    
            else:                  
                nxg.add_node(volume_name,label=f"{volume_name}",shape='image', image = volume_img_url,kind="volume",size=node_size)
                nxg.add_edge(pod_name,volume_name,color=green)        

generate_pod_graph(all_pods_extract_data)

g.from_nx(nxg)
# pp.pprint(g.get_network_data())


nxg2 = nx.Graph()
print(dir(nxg2))

# g.show("output.html")
    

['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__networkx_cache__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adj', '_node', 'add_edge', 'add_edges_from', 'add_node', 'add_nodes_from', 'add_weighted_edges_from', 'adj', 'adjacency', 'adjlist_inner_dict_factory', 'adjlist_outer_dict_factory', 'clear', 'clear_edges', 'copy', 'degree', 'edge_attr_dict_factory', 'edge_subgraph', 'edges', 'get_edge_data', 'graph', 'graph_attr_dict_factory', 'has_edge', 'has_node', 'is_directed', 'is_multigraph', 'name', 'nbunch_iter', 'neighbors', 'node_attr_dict_factory', 'node_dict_factory', 'nodes', 'number_of_edges', 'number_of_nodes', 'order', 'remove_edge', 'remove_edges_from', 'remove_node', 'r

In [8]:
import subprocess

# Run the kubectl command and pipe the output to yq
result = subprocess.run(
    "kubectl get pod ingress-nginx-controller-5fd8d8557c-jsm4b -n ingress-nginx -o yaml | yq eval '.spec.volumes'",
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Decode the output from bytes to string and print it
output = result.stdout.decode()
error = result.stderr.decode()

if result.returncode == 0:
    print(output)
else:
    print("Error:", error)


null

